# Homework solution. Week 3.

In [42]:
import os

import pandas as pd
from google.cloud import storage

## Setup

In [ ]:
DATA_URL = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv"
OUTPUT_DIR = "./data"


def get_file_uri(year, month):
    return os.path.join(DATA_URL, f"fhv_tripdata_{year}-{month:02}.csv.gz")


def download_file(year, month, output_dir: str = None):
    if output_dir is None:
        output_dir = OUTPUT_DIR

    os.makedirs(output_dir, exist_ok=True)

    file_uri = get_file_uri(year, month)
    data = pd.read_csv(file_uri)

    filename = os.path.join(output_dir, os.path.basename(file_uri))
    data.to_csv(filename, index=False)

    return filename


def upload_to_gcs(bucket, object_name, local_file):
    """
    Ref: https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python
    """
    # # WORKAROUND to prevent timeout for files > 6 MB on 800 kbps upload speed.
    # # (Ref: https://github.com/googleapis/python-storage/issues/74)
    # storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB
    # storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB

    client = storage.Client()
    bucket = client.bucket(bucket)
    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)

Download trips data from github to local storage

In [41]:
%%time

YEAR = 2019

for month in range(1, 13):
    print(f"Loading the data... (month: {month})")
    try:
        print(download_file(year=YEAR, month=month))
    except Exception as e:
        print(e)
        continue

Loading the data... (month: 1)
./data/fhv_tripdata_2019-01.csv.gz
Loading the data... (month: 2)
./data/fhv_tripdata_2019-02.csv.gz
Loading the data... (month: 3)
./data/fhv_tripdata_2019-03.csv.gz
Loading the data... (month: 4)
./data/fhv_tripdata_2019-04.csv.gz
Loading the data... (month: 5)
./data/fhv_tripdata_2019-05.csv.gz
Loading the data... (month: 6)
./data/fhv_tripdata_2019-06.csv.gz
Loading the data... (month: 7)
./data/fhv_tripdata_2019-07.csv.gz
Loading the data... (month: 8)
./data/fhv_tripdata_2019-08.csv.gz
Loading the data... (month: 9)
./data/fhv_tripdata_2019-09.csv.gz
Loading the data... (month: 10)
./data/fhv_tripdata_2019-10.csv.gz
Loading the data... (month: 11)
./data/fhv_tripdata_2019-11.csv.gz
Loading the data... (month: 12)
./data/fhv_tripdata_2019-12.csv.gz
CPU times: user 6min 5s, sys: 8.37 s, total: 6min 14s
Wall time: 7min 44s


Setting up credentials

```
export GOOGLE_APPLICATION_CREDENTIALS="/Users/a_kulesh/Workspace/education/courses/data-engineering-zoomcamp/_fork/data-engineering-zoomcamp/virtual-dynamo-375412-88efae140216.json"
```

Refresh token/session, and verify authentication
```
gcloud auth application-default login
```

Upload files to GCS bucket

In [53]:
%%time

BUCKET = "dtc_data_lake_virtual-dynamo-375412"

names = os.listdir(OUTPUT_DIR)
for name in names:
    object_name = os.path.join("data/fhv", name)
    file_name = os.path.join(OUTPUT_DIR, name)

    upload_to_gcs(BUCKET, object_name, file_name)
    print(f"File '{file_name}' uploaded to GCS: '{BUCKET}/{object_name}'")

File './data/fhv_tripdata_2019-11.csv.gz' uploaded to GCS: 'dtc_data_lake_virtual-dynamo-375412/data/fhv/fhv_tripdata_2019-11.csv.gz'
File './data/fhv_tripdata_2019-03.csv.gz' uploaded to GCS: 'dtc_data_lake_virtual-dynamo-375412/data/fhv/fhv_tripdata_2019-03.csv.gz'
File './data/fhv_tripdata_2019-01.csv.gz' uploaded to GCS: 'dtc_data_lake_virtual-dynamo-375412/data/fhv/fhv_tripdata_2019-01.csv.gz'
File './data/fhv_tripdata_2019-09.csv.gz' uploaded to GCS: 'dtc_data_lake_virtual-dynamo-375412/data/fhv/fhv_tripdata_2019-09.csv.gz'
File './data/fhv_tripdata_2019-05.csv.gz' uploaded to GCS: 'dtc_data_lake_virtual-dynamo-375412/data/fhv/fhv_tripdata_2019-05.csv.gz'
File './data/fhv_tripdata_2019-07.csv.gz' uploaded to GCS: 'dtc_data_lake_virtual-dynamo-375412/data/fhv/fhv_tripdata_2019-07.csv.gz'
File './data/fhv_tripdata_2019-10.csv.gz' uploaded to GCS: 'dtc_data_lake_virtual-dynamo-375412/data/fhv/fhv_tripdata_2019-10.csv.gz'
File './data/fhv_tripdata_2019-02.csv.gz' uploaded to GCS: 'dt

Create a dataset in the same location as the source files and create an external table:
```
CREATE OR REPLACE EXTERNAL TABLE `virtual-dynamo-375412.nytaxi.external_tripdata`
OPTIONS (
  format = 'CSV',
  compression = 'GZIP',
  uris = ['gs://dtc_data_lake_virtual-dynamo-375412/data/fhv/fhv_tripdata_2019*']
);
```

Create a table from the external table:
```
CREATE OR REPLACE TABLE virtual-dynamo-375412.nytaxi.tripdata_non_partitioned AS
SELECT * FROM virtual-dynamo-375412.nytaxi.external_tripdata;
```

## Question 1:
What is the count for fhv vehicle records for year 2019?

```
SELECT count(*)
FROM virtual-dynamo-375412.nytaxi.tripdata_non_partitioned
```

> 43244696

## Question 2:
Write a query to count the distinct number of affiliated_base_number for the entire dataset on both the tables.</br> 
What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?

```
SELECT count(distinct(Affiliated_base_number))
FROM virtual-dynamo-375412.nytaxi.tripdata_non_partitioned;

SELECT count(distinct(Affiliated_base_number))
FROM virtual-dynamo-375412.nytaxi.external_tripdata;
```

> - 0 MB for the External Table and 0MB for the BQ Table

## Question 3:
How many records have both a blank (null) PUlocationID and DOlocationID in the entire dataset?

```
select count(*)
from virtual-dynamo-375412.nytaxi.tripdata_non_partitioned
where PUlocationID is null
and DOlocationID is null;
```

> 717748

## Question 4:
What is the best strategy to optimize the table if query always filter by pickup_datetime and order by affiliated_base_number?

> - Partition by pickup_datetime Cluster on affiliated_base_number

## Question 5:
Implement the optimized solution you chose for question 4. Write a query to retrieve the distinct affiliated_base_number between pickup_datetime 2019/03/01 and 2019/03/31 (inclusive).</br> 
Use the BQ table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 4 and note the estimated bytes processed. What are these values? Choose the answer which most closely matches.

---
Create partitioned and cluster table:
```
CREATE OR REPLACE TABLE virtual-dynamo-375412.nytaxi.tripdata_partitioned
PARTITION BY
  DATE(pickup_datetime)
CLUSTER BY Affiliated_base_number AS
SELECT * FROM virtual-dynamo-375412.nytaxi.external_tripdata;
```

Run query for both tables:
```
SELECT count(distinct(Affiliated_base_number))
FROM virtual-dynamo-375412.nytaxi.tripdata_non_partitioned
where pickup_datetime between '2019-03-01' and '2019-03-31';

SELECT count(distinct(Affiliated_base_number))
FROM virtual-dynamo-375412.nytaxi.tripdata_partitioned
where pickup_datetime between '2019-03-01' and '2019-03-31';
```

> - 647.87 MB for non-partitioned table and 23.06 MB for the partitioned table

## Question 6: 
Where is the data stored in the External Table you created?

> - GCP Bucket

## Question 7:
It is best practice in Big Query to always cluster your data:
> - False